# Proyecto
## Álvaro Andrés Esquivel Gómez 11002822

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from joblib import dump, load
from sklearn import datasets
from sklearn.utils import resample
from sklearn import svm
from datetime import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from scipy import stats
import collections

In [2]:
if tf.__version__.startswith("2."):
    import tensorflow.compat.v1 as tf
    tf.compat.v1.disable_v2_behavior()
    tf.compat.v1.disable_eager_execution()
    print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


### Train-val-test split 
El primer paso es separar los datos en entrenamiento, validación y pruebas:

* separar datos en entrenamiento y pruebas. Por ejemplo usando: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
* tomar una porción de datos de entrenamiento del paso anterior para validación. Puede ser aplicando nuevamente https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html


In [3]:
#Importando los datos
data = pd.read_csv (r'data_titanic_proyecto.csv')
data.head()

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N


In [4]:
#Quitando variables que no aportan informacion al modelo
data = data.drop(['PassengerId', 'Fare', 'Name', 'Ticket', 'Cabin'], axis=1)
print(data.shape)
data.head()
#entrenamiento, validacion = train_test_split(data, test_size=0.80, random_state=10)

(891, 7)


,Age,SibSp,Parch,Embarked,passenger_class,passenger_sex,passenger_survived
0,22.0,1,0,S,Lower,M,N
1,38.0,1,0,C,Upper,F,Y
2,26.0,0,0,S,Lower,F,Y
3,35.0,1,0,S,Upper,F,Y
4,35.0,0,0,S,Lower,M,N


In [5]:
#Remplaza valores NaN con 0
data["Age"] = data["Age"].replace(np.nan, 0)
print(data.info())
data.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 891 non-null    float64
 1   SibSp               891 non-null    int64  
 2   Parch               891 non-null    int64  
 3   Embarked            889 non-null    object 
 4   passenger_class     891 non-null    object 
 5   passenger_sex       891 non-null    object 
 6   passenger_survived  891 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 48.9+ KB
None


(891, 7)

In [6]:
#Copia variables categoricas
#data = data.select_dtypes(include=['object']).copy()
#cat_data.head()
#Variables cono datos null
#print(cat_data.isnull().sum())
#Elimina los valores vacios
data = data.fillna(data['Embarked'].value_counts().index[0])
#print(cat_data.isnull().sum())
print(data['Embarked'].value_counts(),"\n", data['passenger_class'].value_counts(),"\n", 
      data['passenger_sex'].value_counts(),"\n", data['passenger_survived'].value_counts())

S    646
C    168
Q     77
Name: Embarked, dtype: int64 
 Lower     491
Upper     216
Middle    184
Name: passenger_class, dtype: int64 
 M    577
F    314
Name: passenger_sex, dtype: int64 
 N    549
Y    342
Name: passenger_survived, dtype: int64


In [7]:
#Remplazando valores categoricos
replace_map = {'Embarked': {'S': 1, 'C': 2, 'Q': 3}, 'passenger_class': {'Lower': 1, 'Middle': 2, 'Upper': 3}, 
               'passenger_sex': {'F': 0, 'M': 1}, 'passenger_survived': {'N': 0, 'Y': 1}}
data.replace(replace_map, inplace=True)

data.head(10)

,Age,SibSp,Parch,Embarked,passenger_class,passenger_sex,passenger_survived
0,22.0,1,0,1,1,1,0
1,38.0,1,0,2,3,0,1
2,26.0,0,0,1,1,0,1
3,35.0,1,0,1,3,0,1
4,35.0,0,0,1,1,1,0
5,0.0,0,0,3,1,1,0
6,54.0,0,0,1,3,1,0
7,2.0,3,1,1,1,1,0
8,27.0,0,2,1,1,0,1
9,14.0,1,0,2,2,0,1


In [8]:
train, test = train_test_split(data, test_size=0.20, random_state = 10)
train, validation = train_test_split(train, test_size=0.20, random_state = 10)
print("Train:", train.shape, "test:", test.shape, "validation:", validation.shape)

Train: (569, 7) test: (179, 7) validation: (143, 7)


#### Ensemble learning:
Se aplicará ensemble learning para crear una "votación mayoritaria" con distintos tipos de modelos, con el objetivo de simplificar el problema 2 de estos modelos se harán usando scikit-learn y su función   .fit(x,y) 


In [9]:
data2 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
# prepare bootstrap sample
boot = resample(data2, replace=True, n_samples=6, random_state=2)
print('Bootstrap Sample: %s' % boot)

Bootstrap Sample: [0.1, 0.6, 0.1, 0.4, 0.3, 0.4]


## Matriz de evalueación

In [38]:
columnas = ['accuracy','porcentaje_error', 'precision_score', 'recall_score', 'f1_score']
evaluation_matrix  = pd.DataFrame(columns=columnas)

evaluation_matrix

,accuracy,porcentaje_error,precision_score,recall_score,f1_score


### Funcion calculo de evaluaciones y agrega a matriz

In [11]:
def agegar_resultados(y_test, y_hat, index_name):

    error_squared = mean_squared_error(y_test, y_hat)
    print("error_squared:", error_squared)
    acc_reg_log = accuracy_score(y_test, y_hat)
    print("accuracy o exactitud", acc_reg_log)
    porcentaje_error = 1 - acc_reg_log
    print("porcentaje de error", porcentaje_error)
    p_score = precision_score(y_test, y_hat, average='binary')
    print("precision_score", p_score)
    r_score = recall_score(y_test, y_hat, average='binary')
    print("recall_score", r_score)
    f_score = f1_score(y_test, y_hat, average='binary')
    print("f1 score", f_score)

    #Agrego al dataframe 
    evaluation_data = pd.DataFrame({"accuracy":acc_reg_log, "porcentaje_error":porcentaje_error, 
                                    "precision_score":p_score, "recall_score":r_score, "f1_score":f_score}, 
                                   index=[index_name]) 
    return evaluation_matrix.append(evaluation_data)

## Árbol de decisión con sklearn

In [56]:
def train_decision_tree(train, test, y):
    
    X_train = train.iloc[:, train.columns != y]
    Y_train = train.iloc[:, train.columns == y]
    #print("X_train\n",X_train, "\n\nY_train\n", Y_train)
    x_test = test.iloc[:, test.columns != y]
    y_test = test.iloc[:, test.columns == y]
    #print("x_test\n",X_train, "\n\y_test\n", Y_train)
    
    #Creacion del modelo de arboles
    classifi = DecisionTreeClassifier(random_state = 0)  
    #Entrenamiento
    classifi.fit(X_train, Y_train) 
    
    #Prediccion 
    y_pred = classifi.predict(x_test).astype(int)
    
    #Fecha y hora del modelo para almacenar
    now = datetime.now()
    fecha_hora = now.strftime("%Y%m%d-%H%M%S")
    
    #String de configuración para cada experimento
    str_experimento = "decision_tree_"+fecha_hora
    
    #Almacena el modelo 
    dump(classifi, str_experimento+".joblib") 
    
    error = mean_squared_error(y_test, y_pred)
    return y_pred, error

#X_train = train.iloc[:, 0:8]
#Y_train = train.iloc[:, 8]


In [57]:
resultado = train_decision_tree(train, test, "passenger_survived")
print(resultado)

(array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 0]), 0.20670391061452514)


In [41]:
y_test = test.iloc[:, test.columns == "passenger_survived"]
evaluation_matrix = agegar_resultados(y_test, resultado[0], 'decision_tree')
evaluation_matrix

error_squared: 0.2122905027932961
accuracy o exactitud 0.7877094972067039
porcentaje de error 0.2122905027932961
precision_score 0.7068965517241379
recall_score 0.6612903225806451
f1 score 0.6833333333333333


,accuracy,porcentaje_error,precision_score,recall_score,f1_score
decision_tree,0.787709,0.212291,0.706897,0.66129,0.683333


## SVM con sklearn

In [53]:
def train_SVM(train, test, y):
    
    X_train = train.iloc[:, train.columns != y]
    Y_train = train.iloc[:, train.columns == y].to_numpy().flatten()
    x_test = test.iloc[:, test.columns != y]
    y_test = test.iloc[:, test.columns == y].to_numpy().flatten()
    

    #Modelo SVM
    clf = svm.SVC(kernel='linear') 

    #Entrenamiento
    clf.fit(X_train, Y_train)
    #Prediccion
    y_pred = clf.predict(x_test)
    
    #Fecha y hora del modelo para almacenar
    now = datetime.now()
    fecha_hora = now.strftime("%Y%m%d-%H%M%S")
    
    #String de configuración para cada experimento
    str_experimento = "SVM_sklearn_"+fecha_hora
    
    #Almacena el modelo 
    dump(clf, str_experimento+".joblib") 
    
    error = mean_squared_error(y_test, y_pred)
    return y_pred, error


In [51]:
resultado = train_SVM(train, test, "passenger_survived")
resultado

(array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0], dtype=int64),
 0.1787709497206704)

In [43]:
y_test = test.iloc[:, test.columns == "passenger_survived"]
evaluation_matrix = agegar_resultados(y_test, resultado[0], 'SVM')
evaluation_matrix

error_squared: 0.1787709497206704
accuracy o exactitud 0.8212290502793296
porcentaje de error 0.17877094972067042
precision_score 0.7678571428571429
recall_score 0.6935483870967742
f1 score 0.7288135593220338


,accuracy,porcentaje_error,precision_score,recall_score,f1_score
decision_tree,0.787709,0.212291,0.706897,0.661290,0.683333
SVM,0.821229,0.178771,0.767857,0.693548,0.728814


## Naive bayes con numpy y pandas

In [18]:
X_train = train.iloc[:, train.columns != "passenger_survived"].to_numpy()
Y_train = train.iloc[:, train.columns == "passenger_survived"].to_numpy()
Y_train = Y_train[:, 0]
x_test = test.iloc[:, test.columns != "passenger_survived"].to_numpy()
y_test = test.iloc[:, test.columns == "passenger_survived"].to_numpy()
Y_train

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0,

In [19]:
#Calculo de prior probability
def prior_probability(y):
    y_dict = collections.Counter(y)
    pre_probab = np.ones(2)
    for i in range(0, 2):
        pre_probab[i] = y_dict[i]/y.shape[0]
    return pre_probab

In [20]:
#Funcion de media de las features 
def mean_var(X, y):
    n_features = X.shape[1]
    m = np.ones((2, n_features))
    v = np.ones((2, n_features))
    n_0 = np.bincount(y)[np.nonzero(np.bincount(y))[0]][0] #<<<-----
    x0 = np.ones((n_0, n_features))
    x1 = np.ones((X.shape[0] - n_0, n_features))
    k = 0
    for i in range(0, X.shape[0]):
        if y[i] == 0:
            x0[k] = X[i]
            k = k + 1
    k = 0
    for i in range(0, X.shape[0]):
        if y[i] == 1:
            x1[k] = X[i]
            k = k + 1
    for j in range(0, n_features):
        m[0][j] = np.mean(x0.T[j])
        v[0][j] = np.var(x0.T[j])*(n_0/(n_0 - 1))
        m[1][j] = np.mean(x1.T[j])
        v[1][j] = np.var(x1.T[j])*((X.shape[0]-n_0)/((X.shape[0]
                                                      - n_0) - 1))
    return m, v


In [21]:
def prob_feature_class(m, v, x):
    n_features = m.shape[1]
    #print(n_features)
    pfc = np.ones(2)
    #print(pfc)
    for i in range(0, 2):
        product = 1
        for j in range(0, n_features):
            #print(1/np.sqrt(2*3.14*v[i][j]))
            #print(1/np.sqrt(2*3.14*v[i][j]))
            product = product * (1/np.sqrt(2*math.pi*v[i][j])) * math.exp(-0.5
                                 * pow((x[j] - m[i][j]),2)/v[i][j])
            #print(x)
        pfc[i] = product
    return pfc


In [22]:
def naive_bayes(X, y, x):
    m, v = mean_var(X, y)
    pfc = prob_feature_class(m, v, x)
    pre_probab = prior_probability(y)
    pcf = np.ones(2)
    total_prob = 0
    for i in range(0, 2):
        total_prob = total_prob + (pfc[i] * pre_probab[i])
    for i in range(0, 2):
        pcf[i] = (pfc[i] * pre_probab[i])/total_prob
    prediction = int(pcf.argmax())
    return m, v, pre_probab, pfc, pcf, prediction

In [52]:
def train_naive_bayes(train, test, y):
    
    X_train = train.iloc[:, train.columns != y].to_numpy()
    Y_train = train.iloc[:, train.columns == y].to_numpy()
    Y_train = Y_train[:, 0]
    x_test = test.iloc[:, test.columns != y].to_numpy()
    y_test = test.iloc[:, test.columns == y].to_numpy()

    pred = np.empty(0)
    for i in range(x_test.shape[0]):
        x = np.array(x_test[i])
        #print(x)
        m, v, pre_probab, pfc, pcf, prediction = naive_bayes(X_train, Y_train, x)
        #print(prediction)
        pred = np.append(pred, prediction)

    #print(y_test.flatten())
    error = mean_squared_error(y_test, pred)
    return pred, error

resultado = train_naive_bayes(train, test, "passenger_survived")
resultado

(array([0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
        0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0.,
        1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
        0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
        0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 1.,
        0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
        0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.,
        1., 0., 0., 1., 1., 0., 0., 0., 1.]),
 0.24022346368715083)

In [45]:
y_test = test.iloc[:, test.columns == "passenger_survived"]
evaluation_matrix = agegar_resultados(y_test, resultado[0], 'naive_bayes')
evaluation_matrix

error_squared: 0.24022346368715083
accuracy o exactitud 0.7597765363128491
porcentaje de error 0.24022346368715086
precision_score 0.6301369863013698
recall_score 0.7419354838709677
f1 score 0.6814814814814814


,accuracy,porcentaje_error,precision_score,recall_score,f1_score
decision_tree,0.787709,0.212291,0.706897,0.661290,0.683333
SVM,0.821229,0.178771,0.767857,0.693548,0.728814
naive_bayes,0.759777,0.240223,0.630137,0.741935,0.681481


## Reg. logística binaria(sigmoid)  en Tensorflow con regularización

In [25]:
#Funcion de error o de costo del modelo

def error(y_real,y_aprox):
    return tf.nn.sigmoid_cross_entropy_with_logits(labels = y_real, logits = y_aprox)

In [30]:
#Funcion de entrenamiento de regresion logistica
def train_log_reg(x, y, x_test, y_test, n, lr, epochs, n_visualizaciones):

    #Reinicio del grafo
    tf.reset_default_graph()
    
    with tf.name_scope("Hiperparametros"):
        #Hiperparametros
        learning_rate = lr
        training_epochs = epochs
    
    with tf.name_scope("Parametros_entrenables"):
        #Parametros entrenables del modelo iniciando en 0,0
        par_entrenables = tf.Variable(tf.zeros([n, 2]), 
                                      dtype=tf.float32, name = "parametros_entrenables",)
        l_rate = tf.get_variable("l_rate",dtype=tf.float32, shape=[], 
                                 initializer=tf.zeros_initializer())
        b = tf.Variable(tf.zeros([2])) 

    with tf.name_scope("Placeholders"):
        #Datos para el modelo, tam_muesta = total de elementos que tiene el modelo 
        tensor_x = tf.placeholder(tf.float32,[None, n],"tensor_x")
        tensor_y = tf.placeholder(tf.float32,[None, 2],"tensor_y")
    
    with tf.name_scope("Prediccion_y-hat"):
        y_hat = tf.nn.sigmoid(tf.add(tf.matmul(tensor_x, par_entrenables), b), name = "predicciones-y_hat")
    
    with tf.name_scope("ErrorMedio"):
        val_error = tf.reduce_mean(error(tensor_y,y_hat), name = "Calculo_error")
        es_error = tf.summary.scalar(name='es_error', tensor=val_error)
    
    with tf.name_scope("Gradientes"):
        gradiente = tf.gradients(error(tensor_y,y_hat),par_entrenables, name = "Calculo_gradientes")
    
    with tf.name_scope("Nuevos-Parametros"):
        actualizacion_parametros = tf.assign(par_entrenables, tf.reshape(
            tf.math.subtract(par_entrenables, (l_rate * gradiente)), [n, 2]), 
                                             name = "Actualizacion_parametros")
        
    #Crea una sesion
    with tf.train.MonitoredSession() as session:
        #Agrega las observaciones a un vextor con valores: 1, x1, x2...

        #Diccionario de entrada con valores de X y Y del modelo
        #Envia todo el set de datos al modelo
        feed_dict = {tensor_x:x, tensor_y:y, l_rate:learning_rate}
        
        #Logs con TensorBoard
        now = datetime.now()
        fecha_hora = now.strftime("%Y%m%d-%H%M%S")
        #String de configuración para cada experimento
        path = "./logs/train_log_reg_"+fecha_hora + "_lr=" + str(lr) + "_epochs=" +  str(epochs)
        print("String experimento:", path)
        
        #Imprime grafo en TensorBoard
        writer = tf.summary.FileWriter(path, session.graph)
        
        #Interacion por cada epoch, segun fue definido
        for i in range(training_epochs):

            #Imprime los valores del error cada cierta cantidad de interaciones
            if (i) % n_visualizaciones == 0: 
                #Derivadas parciales del error 
                session.run(gradiente,feed_dict=feed_dict)
                s = session.run(par_entrenables,feed_dict=feed_dict)
                print("Ejecucion:", i, "\nerror: ", session.run(val_error,feed_dict=feed_dict))
            else:
                session.run(gradiente,feed_dict=feed_dict)
            
            #Error a TensorBoard
            summary = session.run(es_error,feed_dict=feed_dict)
            writer.add_summary(summary, i)
            
            #Actualiza a los nuevos parametros
            session.run(actualizacion_parametros,feed_dict=feed_dict)

            #Predicciones segun los nuevos parametros
            predicciones = session.run(y_hat,feed_dict=feed_dict)
            
        #Almacenar parametros de prediccion
        parametros_entre = session.run(par_entrenables,feed_dict=feed_dict)
        str_experimento = "log_reg_"+fecha_hora + "_lr=" + str(lr) + "_epochs=" +  str(epochs)
        np.save(str_experimento, parametros_entre)
        
        #Prediccion final con test
        feed_dict = {tensor_x:x_test, tensor_y:y_test, l_rate:learning_rate}
        pred = session.run(y_hat,feed_dict=feed_dict)
        writer.close()
        return pred



### Grafico del grafo

![title](img/grafo-train_log_reg_20200629.png)

In [31]:
X_data = data.iloc[:, data.columns != "passenger_survived"]
Y_data = data.iloc[:, data.columns == "passenger_survived"]   
#One hot
oneHot = OneHotEncoder() 
#print("X_train\n", X_train, X_train.shape)
#print("Y_train\n", Y_train, Y_train.shape)
# Encoding x_orig 
oneHot.fit(X_data) 
X_data = oneHot.transform(X_data).toarray() 
  
# Encoding y_orig 
oneHot.fit(Y_data) 
Y_data = oneHot.transform(Y_data).toarray() 
print("X_data\n", X_data, X_data.shape)
print("Y_data\n", Y_data, Y_data.shape)

x_train, x_test = train_test_split(X_data, test_size=0.20, random_state = 10)
x_train, x_validation = train_test_split(x_train, test_size=0.20, random_state = 10)
print("x_train:", x_train.shape, "x_test:", x_test.shape, "x_validation:", x_validation.shape)

y_train, y_test = train_test_split(Y_data, test_size=0.20, random_state = 10)
y_train, y_validation = train_test_split(y_train, test_size=0.20, random_state = 10)
print("y_train:", y_train.shape, "y_test:", y_test.shape, "y_validation:", y_validation.shape)
m, n = x_train.shape

#print("\n\n\nx_train", x_train, "\n\ny_train", y_train, "\n\nx_test", x_test, "\n\ny_test", y_test, "\nn",n)
predic = train_log_reg(x_train, y_train, x_test, y_test, n, 0.01, 50, 2)


X_data
 [[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]] (891, 111)
Y_data
 [[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [1. 0.]] (891, 2)
x_train: (569, 111) x_test: (179, 111) x_validation: (143, 111)
y_train: (569, 2) y_test: (179, 2) y_validation: (143, 2)
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
String experimento: ./logs/train_log_reg_20200629-180809_lr=0.01_epochs=50
Ejecucion: 0 
error:  0.7240757
Ejecucion: 2 
error:  0.6718648
Ejecucion: 4 
error:  0.65704036
Ejecucion: 6 
error:  0.6479404
Ejecucion: 8 
error:  0.6413755
Ejecucion: 10 
error:  0.6362801
Ejecucion: 12 
error:  0.6322115
Ejecucion: 14 
error:  0.6289329
Ejecucion: 16 
error:  0.62626886
Ejecucion: 18 
error:  0.62407637
Ejecucion: 20 
error:  0.6222442
Ejecucion: 22 
error:  0.62068844
Ejecucion: 24 
error:  0.61934906
Ej

![title](img/error-train_log_reg_20200629.png)

In [46]:
#print(y_test)
predic[0:predic.shape[0],]
resultado = []
for i in range(predic.shape[0]):
    if predic[i, 0] > predic[i, 1]:
        resultado.append(0)
    else:
        resultado.append(1)

print(resultado)
Y_data = data.iloc[:, data.columns == "passenger_survived"]  
y_train, y_test = train_test_split(Y_data, test_size=0.20, random_state = 10)
y_train, y_validation = train_test_split(y_train, test_size=0.20, random_state = 10)


evaluation_matrix = agegar_resultados(y_test, resultado, 'reg_logistica')
evaluation_matrix


[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
error_squared: 0.17318435754189945
accuracy o exactitud 0.8268156424581006
porcentaje de error 0.17318435754189943
precision_score 0.8297872340425532
recall_score 0.6290322580645161
f1 score 0.7155963302752293


,accuracy,porcentaje_error,precision_score,recall_score,f1_score
decision_tree,0.787709,0.212291,0.706897,0.661290,0.683333
SVM,0.821229,0.178771,0.767857,0.693548,0.728814
naive_bayes,0.759777,0.240223,0.630137,0.741935,0.681481
reg_logistica,0.826816,0.173184,0.829787,0.629032,0.715596


# K-Folds Cross Validation

K-Fold Cross Validation busca resolver el treade-off entre el utilizar el dataset completo para un entrenameinto mas efectico (con mayor cantidad de datos), y tener una buena muestra del dataset para su posterior evaluación. 

La idea es utilizar todo el dataset para el entrenamiento y todo el dataset para la validación, teniendo en cuenta que cuando realizamos validación siempre debe de ser con datos con los que no fue entrenado el modelo. 


![title](img/ejemplo_cross_validation.png)

La imagen anterior nos muestra como funcionaria Cross Validation utilizando un k = 5, el algoritmo seria el siguiente:

1. Tomamos todo el dataset y este se divide en 5 partes de tamaño x.
2. Guardamos la primer parte de la division para el proceso de validación, y ejecutamos el proceso de entrenamiento con las otras 4 partes restantes.
3. Una vez entrenado el modelo, utilizamos la primer parte del dataset para validarlo, y obtenemos un resultado. 
4. Repetir el proceso anterior tomando cada vez una parte diferente para el proceso de validación, y las otras 4 restantes para el entrenamiento. 
5. Para definir una metrica de evaluación entre todos los procesos de entrenamiento, promediamos el valor de cada validación realizada.

K-Folds Cross Validation, definitivamente ubiera sido de gran utilidad para esta preactica, ya que tenemos una mejor idea de como se comportarían nuestros modelo, utilizando todos los datos del dataset para el aprendizaje y la evaluación. 

##  Matriz de evaluación de los modelos

In [47]:
evaluation_matrix

,accuracy,porcentaje_error,precision_score,recall_score,f1_score
decision_tree,0.787709,0.212291,0.706897,0.661290,0.683333
SVM,0.821229,0.178771,0.767857,0.693548,0.728814
naive_bayes,0.759777,0.240223,0.630137,0.741935,0.681481
reg_logistica,0.826816,0.173184,0.829787,0.629032,0.715596


# Conclusiones 

* A partir de la metriz de evaluacion de los modelos generados, se puede estimar mediate la metrica de f1 score, que los modelos de regresión logistica ejecutado en Tensorflow y el modelo Support Vector Machines SVM, de sklearn, fueron los que mejor resultado tuvieron, si se debe de elegir solamente uno, creo que el de SVM sería el mas adecuado para la prediccion de datos de este modelo.
* La matriz de evaluación es muy util para estimar la evaluación de los modelos, a partir de ella se pueden priorizar modelos, según sea la prioridad de estudiar. Si solo me interesaría la evaluacion de las predicciones correctas, con la regresión logistica de tensorflow seria suficiente, ya que cuenta con un 82.6% de accuracy. Si lo que necesitamos es reducir los falsos negarivos, el recall score de naive bayes es el adecuado. 


# Recomendaciones

* A partir de la investifacion de Cross Validation y la evaluación de los modelos ejecutados, creo que seria muy conveniente aplicar K-Folds Cross Validation para tener una metrica mas clara de las metricas de los modelos, sobre todo en modelos pequeños como el actual, conde una ejecucción de k aprendizajes no representaria tanto tiempo de aprendizaje. 
* Es recomendado determinar uns prioridad especifica para las metricas de evaluación del modelo, con la finalidad de centrar el resultado de los modelos en esta metrica y proveer un camino de optimización para el aprendizaje del modelo. 


# Experiencia, dificultades y lecciones aprendidas

* Creo que la major experiencia es la del almecenamiento del modelo entrenado para poder producir predicciones en otro notebook, creo que fue lo mas importante para el futuro, ya que en la realidad creo que es así como normalmente se deberia de trabajar, en un sistema especifico de entrenamiento y otro diferente de aprendizaje. 
* La mayor dificultad fue la de realizar naive bayes de forma vectorizada y efectiva, se me dificulto demasiado entender como realizarlo, a pesar de que el algoritmo en sí es muy fácil.
* Realmente son muchas las lecciones aprendidas, sobre todo como comentaba anteriormente, el alamcenamiento del modelo para ejecutarlo en otro entorno, el algoritmo de F-Folds, y las distintas metricas de evaluación, que se aplican según la importancia de lo que se desea evaluar, o hacia donde se debe de ajustar el modelo predictivo. 
